In [16]:
%pip install pythonds3


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import timeit
from pythonds3.basic import Queue
import copy

In [2]:
class Vertex():
    def __init__(self, key):
        self.key = key
        self.num_edges=0
        self.neighbours = {}
        self.distance = 0
        self.previous = None
        self.color = 'white'
    #get weight of the edge from this vertex to the other vertex
    def getEdgeWeight(self, to_vertex):
        return self.neighbours[to_vertex]
    #add a vertex with edge weight 1
    def setNeighbour(self, nbd_vert, bidirect=False):
        setNeighbour(nbd_vert, 1, bidirect)
    #add a vertex with a given edge weight
    def setNeighbour(self, nbd_vert, weight, bidirect=False):
        self.neighbours.update({nbd_vert:weight})
        self.num_edges += 1
        if bidirect:
            nbd_vert.setNeighbour(self, weight)
    #return the list of neighbouring vertices
    #Note: vertices are returned not the key values of the vertices
    def getNeighbours(self):
        return self.neighbours.keys()
    #return the list of key values of neighbouring vertices
    def getNeighboursKeys(self):
        return [x.key for x in self.neighbours]
    #get the key value of this vertex
    def getKey(self):
        return self.key

In [3]:
class Graph():
    def __init__(self, bidirect=False):
        self.vertices = {}
        self.bidirect = bidirect
    #add a vertex with the given key value
    def setVertex(self, key):
        self.vertices.update({key:Vertex(key)})
    #set the edge from one vertex to another with the given weight
    def setEdge(self, from_vertex_key, to_vertex_key, weight=1):
        #not self.vertices[from_vertex_key]:
        if from_vertex_key not in self.vertices:
            self.setVertex(from_vertex_key)
        #not self.vertices[to_vertex_key]:
        if to_vertex_key not in self.vertices:
            self.setVertex(to_vertex_key)
        self.vertices[from_vertex_key].setNeighbour(self.vertices[to_vertex_key], weight, self.bidirect)
    #return vertex with the given key value
    def getVertex(self, vertex_key):
        return self.vertices[vertex_key]
    #return the list of all vertices in the graph
    def getVertexKeys(self):
        return self.vertices.keys()
    #check if there is a vertex with the given key value
    def containsVertex(self, vertex_key):
        return vertex_key in self.vertices

In [4]:
with open("four_letter_words.txt", encoding="utf8") as fin:
    #read the file as a single string and split the lines at newline symbol
    word_list = fin.read().splitlines()

In [5]:
#create categories such that words in each category differ from each other in one letter
buckets = {}

for word in word_list:
    for i,_ in enumerate(word):
        #create labels for word. Four labels per word
        bucket = word[:i]+str("_")+word[i+1:]
        #if the bucket key exist add word to that key
        #if the key does not exist create new key and store word in that key
        buckets.setdefault(bucket, set()).add(word)

In [6]:
buckets

{'_AHS': {'AAHS', 'DAHS', 'FAHS', 'HAHS', 'LAHS', 'PAHS', 'RAHS', 'YAHS'},
 'A_HS': {'AAHS'},
 'AA_S': {'AAHS', 'AALS'},
 'AAH_': {'AAHS'},
 '_ALS': {'AALS', 'BALS', 'DALS', 'GALS', 'MALS', 'PALS', 'SALS'},
 'A_LS': {'AALS', 'AILS', 'ALLS', 'AWLS'},
 'AAL_': {'AALS'},
 '_BAC': {'ABAC'},
 'A_AC': {'ABAC'},
 'AB_C': {'ABAC'},
 'ABA_': {'ABAC', 'ABAS'},
 '_BAS': {'ABAS', 'OBAS'},
 'A_AS': {'ABAS',
  'AGAS',
  'AIAS',
  'AKAS',
  'ALAS',
  'AMAS',
  'ANAS',
  'AUAS',
  'AVAS'},
 'AB_S': {'ABAS', 'ABBS', 'ABOS', 'ABYS'},
 '_BBA': {'ABBA'},
 'A_BA': {'ABBA', 'ALBA', 'ARBA'},
 'AB_A': {'ABBA'},
 'ABB_': {'ABBA', 'ABBE', 'ABBS'},
 '_BBE': {'ABBE'},
 'A_BE': {'ABBE', 'ALBE'},
 'AB_E': {'ABBE', 'ABLE', 'ABYE'},
 '_BBS': {'ABBS', 'EBBS'},
 'A_BS': {'ABBS', 'ALBS', 'ARBS'},
 '_BED': {'ABED'},
 'A_ED': {'ABED',
  'ACED',
  'AGED',
  'AHED',
  'AKED',
  'APED',
  'ARED',
  'AWED',
  'AXED'},
 'AB_D': {'ABED', 'ABID'},
 'ABE_': {'ABED', 'ABET'},
 '_BET': {'ABET', 'YBET'},
 'A_ET': {'ABET', 'ARET'},
 

In [7]:
#create an undirected graph
word_graph = Graph(bidirect=True)

In [8]:
#create vertex for each word
for word in word_list:
    word_graph.setVertex(word)
#create edges between words that are in the same category
for similar_words_key in buckets:
    for word1 in buckets[similar_words_key]:
        for word2 in buckets[similar_words_key]-{word1}:
            word_graph.setEdge(word1, word2)

In [19]:
word_graph.getVertex("WOOD").getNeighboursKeys()

['MOOD',
 'FOOD',
 'HOOD',
 'POOD',
 'GOOD',
 'ROOD',
 'WOLD',
 'WORD',
 'WOAD',
 'WOON',
 'WOOL',
 'WOOT',
 'WOOS',
 'WOOF']

In [20]:
def searchShortestPath(graph, from_word, to_word):
    start = graph.getVertex(from_word)
    start.distance = 0
    start.previous = None
    end = graph.getVertex(to_word)
    vertex_queue = Queue()
    vertex_queue.enqueue(start)
    current = start
    while bool(vertex_queue.size()):
        current = vertex_queue.dequeue()
        if current.key != end.key:
            for neighbour in current.getNeighbours():
                if neighbour.color == "white":
                    neighbour.color = 'gray'
                    neighbour.previous = current
                    neighbour.distance = current.distance+1
                    vertex_queue.enqueue(neighbour)
            current.color = 'black'
    path = [end]
    current = end
    while current.distance!=0:
        path.append(current.previous)
        current = current.previous
    for x in graph.vertices.values():
        x.distance = 0
        x.color = "white"
        x.previous = None
    return path

In [25]:
path = searchShortestPath(word_graph, "IDLE", "BUSY")

In [26]:
path = [x.key for x in path][::-1]
path

['IDLE', 'IDEE', 'IDES', 'ODES', 'ODDS', 'OUDS', 'BUDS', 'BUSS', 'BUSY']